<a href="https://colab.research.google.com/github/SashaNasonova/burnSeverity/blob/2025-updates/BurnSeverityMapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Burn Severity Mapping Notebook
This notebook is intended to be used for small scale, interactive burn severity mapping of individual fires. For large scale semi-automated mapping please refer to the main python scripts (https://github.com/SashaNasonova/burnSeverity).

The methodology is based on the Burned Area Reflectance Classification (BARC) product developed by the USGS that aims to estimate burn severity through a spectral comparison of pre- and post-fire medium resolution (20 - 30m) satellite imagery.

Healthy vegetation reflects strongly in the near-infrared (NIR) portion of the electromagnetic spectrum whereas rock and bare soil reflects strongly in the mid to shortwave infrared (SWIR) portion. In other words, healthy vegetation reflects strongly in NIR and reflects weakly in SWIR **(↑NIR,↓SWIR)**, whereas soil, bare rock and burned woody vegetation reflect strongly in SWIR and weakly in NIR **(↑SWIR,↓NIR)**. This inverse relationship can be leveraged to provide an estimate of burn severity where both pre- and post-fire imagery is available.

The Normalized Burn Ratio (NBR) is a spectral index that captures the relationship between NIR and SWIR bands. The difference between pre- and post-fire NBR (dNBR) can then be used to quantify wildfire burn severity (**↑dNBR ∝ ↑Severity**) using the following equations.

(1) NBR = (NIR - SWIR) / (NIR + SWIR) \\
(2) dNBR = NBRpre - NBRpost

Once dNBR has been calculated, it can be transformed into a burn severity classification product using a variety of methods ranging from simple thresholding to more complex supervised classifications informed by ground observations. This process is based on the USGS BARC256 methodology which scales the data to an 8-bit representation and utilizes static thresholds (76,110,187) to create a burn severity classification from the dNBR raster.

This notebook is divided into the following sections:

1. Set up (data import, package installation, library loading)
2. Google Earth Engine authentication and initialization
3. Fire perimeter import and visualization
4. Individual fire perimeter selection (1 fire perimeter)
5. Sensor selection
6. Pre-fire image search, visualization and selection
7. Post-fire image search, visualization and selection
8. BARC mapping
9. Final visualization for quality control
10. Image download
11. Quicklooks and area burned by severity class
12. Final export

Before beginning, please ensure that you are registered to use Google Earth Engine and have the Google Earth Engine API enabled as part of a Google Cloud project. If that is not the case, please follow the instructions on getting started (https://github.com/SashaNasonova/burnSeverity/blob/main/Getting_Started_with_GEE.md). You will need to take note of the project id and enter it later on.



---



### **Part 1: Set-up**

In [2]:
# Clone github repository to be able to access the test data and provincial extent vector data
!git clone https://github.com/SashaNasonova/burnSeverity.git

Cloning into 'burnSeverity'...
remote: Enumerating objects: 525, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 525 (delta 75), reused 36 (delta 16), pack-reused 386 (from 1)
Receiving objects: 100% (525/525), 270.06 MiB | 19.40 MiB/s, done.
Resolving deltas: 100% (248/248), done.
Updating files: 100% (236/236), done.


In [3]:
# Install the libraries
%pip install geemap #removed version specification
%pip install pycrs rasterio python-pptx cartopy


In [4]:
# Import the libraries
import ee
import geemap
print('geemap version',geemap.__version__)
import os, json, shutil
import geopandas
from osgeo import gdal
from google.colab import files

import rasterio
from rasterio.plot import show
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import numpy as np
import pandas as pd
from pathlib import Path

import cartopy.crs as ccrs
import cartopy.feature as cfeature

from pptx import Presentation
from pptx.util import Cm, Inches
from pptx.util import Pt

geemap version 0.35.2


### Define functions
Please inspect the hidden cells by clicking on the sideways arrow by the Define Functions heading and run. The functions were hidden to improve the readibility of the notebook. The hidden cells contain the processing functions for mosaicking, calculating NBR, tiling for export as well as visualization and quality control.

In [5]:
## Define functions
# Helper function to get files, non-recursive
def getfiles(d,ext):
  paths = []
  for file in os.listdir(d):
      if file.endswith(ext):
          paths.append(os.path.join(d, file))
  return(paths)

# Helper function to get image acquisition date and format into ("yyyy-mm-dd")
def getDate(im):
  return(ee.Image(im).date().format("YYYY-MM-dd"))

# Helper function to get scene ids
def getSceneIds(im):
  return(ee.Image(im).get('PRODUCT_ID'))

# Functions to mosaic by image date
def mosaicByDate(indate):
  d = ee.Date(indate)
  #print(d)
  im = col.filterBounds(poly).filterDate(d, d.advance(1, "day")).mosaic()
  #print(im)
  return(im.set("system:time_start", d.millis(), "system:index", d.format("YYYY-MM-dd")))

def runDateMosaic(col_list):
  #get a list of unique dates within the list
  date_list = col_list.map(getDate).getInfo()
  udates = list(set(date_list))
  udates.sort()
  udates_ee = ee.List(udates)

  #mosaic images by unique date
  mosaic_imlist = udates_ee.map(mosaicByDate)
  return(ee.ImageCollection(mosaic_imlist))

# Calculate NBR using Sentinel-2 imagery
def NBR_S2(image):
  nbr = image.expression(
      '(NIR - SWIR) / (NIR + SWIR)', {
          'NIR': image.select('B8'),
          'SWIR': image.select('B12')}).rename('nbr')
  return(nbr)

# Calculate NBR using Landsat imagery
# Handles Landsat-5, 7, 8 and 9
def NBR_Landsat(image,dattype):
  if (dattype == 'L5')|(dattype == 'L7'):
      nbr = image.expression(
          '(NIR - SWIR) / (NIR + SWIR)', {
              'NIR': image.select('SR_B4'),
              'SWIR': image.select('SR_B7')}).rename('nbr')
  elif (dattype == 'L8')|(dattype == 'L9'):
      nbr = image.expression(
          '(NIR - SWIR) / (NIR + SWIR)', {
              'NIR': image.select('SR_B5'),
              'SWIR': image.select('SR_B7')}).rename('nbr')
  else:
      print('Incorrect Landsat sensor specified')
  return(nbr)

# Tiling function, uses a geometry (footprint) to split into a defined
# number or rows and columns (nx,ny)
def grid_footprint(footprint,nx,ny):
  from shapely.geometry import Polygon, LineString, MultiPolygon
  from shapely.ops import split

  #polygon = footprint
  polygon = Polygon(footprint['coordinates'][0])
  #polygon = Polygon(footprint)

  minx, miny, maxx, maxy = polygon.bounds
  dx = (maxx - minx) / nx  # width of a small part
  dy = (maxy - miny) / ny  # height of a small part

  horizontal_splitters = [LineString([(minx, miny + i*dy), (maxx, miny + i*dy)]) for i in range(ny)]
  vertical_splitters = [LineString([(minx + i*dx, miny), (minx + i*dx, maxy)]) for i in range(nx)]
  splitters = horizontal_splitters + vertical_splitters

  result = polygon
  for splitter in splitters:
      result = MultiPolygon(split(result, splitter))

  coord_list = [list(part.exterior.coords) for part in result.geoms]

  poly_list = []
  for cc in coord_list:
      p = ee.Geometry.Polygon(cc)
      poly_list.append(p)
  return(poly_list)

# Applies scaling functors for surface reflectance Landsat imagery
def apply_scale_factors_ls(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

# Multiplies Sentinel-2 imagery by 0.0001
def apply_scale_factors_s2(image):
  opticalBands = image.select('B.*').multiply(0.0001)
  return image.addBands(opticalBands, None, True)

### QA Functions ###
def ql_3band(outshp,imgpath,outpath):
  # Get fire vector
  dfs_sub = geopandas.read_file(outshp)

  # Read 8-bit image
  src = rasterio.open(imgpath)

  # Create pre-fire quicklook image with burned area boundary overlay
  fig, ax = plt.subplots(figsize=(8, 8)) #10,13
  base = show(src,ax=ax)
  dfs_sub.plot(ax=base, edgecolor='purple', facecolor='none',linewidth=2)
  ax.axis('off')
  plt.savefig(outpath, bbox_inches='tight', dpi=300)

  plt.close()
  src = None

def generate_legend_labels(arr):
  #sort array
  arr.sort()

  default_labels = {
      0: {"black":"Unknown"},
      1: {"gray":"Unburned"},
      2: {"yellow":"Low"},
      3: {"orange":"Medium"},
      4: {"red":"High"}
  }

  legend_labels = {}

  for value in arr:
      #print(default_labels[value]) #for debug
      legend_labels.update(default_labels[value])

  return(legend_labels)


def ql_barc(outshp,barcpath,imgpath,outpath):
  dfs_sub = geopandas.read_file(outshp)

  # Read in BARC classification
  src = rasterio.open(barcpath)
  unique_values = np.unique(src.read(1))

  # Read in post-fire image
  src1 = rasterio.open(imgpath)

  #drop no data 9
  arr = unique_values[unique_values!=9]

  # Create cmap dictionary
  colormap_dict = {0: 'black',
                    1: 'gray',
                    2: 'yellow',
                    3: 'orange',
                    4: 'red'}

  colormap_unique = [colormap_dict[value] for value in arr]
  colormap = matplotlib.colors.ListedColormap(colormap_unique)

  # Create pre-fire quicklook image with burned area boundary overlay
  fig, ax = plt.subplots(figsize=(8, 8))
  background = show((src1,1),ax=ax,cmap='gray')
  base = show(src,cmap=colormap,ax=ax)
  dfs_sub.plot(ax=base, edgecolor='purple', facecolor='none',linewidth=2)
  ax.axis('off')

  legend_labels = generate_legend_labels(arr)

  patches = [Patch(color=color, label=label)
              for color, label in legend_labels.items()]

  ax.legend(handles=patches,
            bbox_to_anchor=(1.05, 1),loc='upper left',
            borderaxespad=0., facecolor="white")


  plt.savefig(outpath, bbox_inches='tight', dpi=300)

  plt.close()
  src = None
  src1 = None

# Generate quicklooks
def generate_ql(outshp,tif_file,qcdir):
  name = Path(tif_file).stem + '.png'
  ql = os.path.join(qcdir,name)
  ql_3band(outshp,tif_file,ql)
  return(ql)

def create_ppt(pptpath):
  prs = Presentation()
  prs.slide_width = Inches(16)
  prs.slide_height = Inches(9)

  prs.save(pptpath)

def add_slide(pptpath,i,j,k,l,df):
  prs = Presentation(pptpath)
  title_only_slide_layout = prs.slide_layouts[5]
  slide = prs.slides.add_slide(title_only_slide_layout)
  title = slide.shapes.title
  title.width = Cm(35)
  title.height = Cm(3.3)
  title.top = Cm(0)
  title.left = Cm(0)

  #Add title
  name = Path(k).stem
  title.text = name

  #First image (pre-fire)
  slide.shapes.add_picture(
      i, left=Cm(0.44), top=Cm(8.2), width=Cm(12.24), height=None
  )
  #Second image (post_fire)
  slide.shapes.add_picture(
      j, left=Cm(12.78), top=Cm(8.2), width=Cm(12.24), height=None
  )

  #get image height
  picture = slide.shapes[1] #second shape pre-img, first is the title
  height_cm = picture.height.cm
  #print(height_cm)

  #Third image (barc)
  slide.shapes.add_picture(
      k, left=Cm(25.02), top=Cm(8.2), width=None, height=Cm(height_cm)
  )

  #Fourth image (location map)
  slide.shapes.add_picture(
      l, left=Cm(32.5), top=Cm(0), width=None, height=Cm(7)
  )

  #add table
  rows, cols = df.shape
  left = Cm(0.9)
  top = Cm(3.2)
  width = Cm(16.3)
  height = Cm(3.75)

  #add table
  shape = slide.shapes.add_table(rows + 1, cols, left, top, width, height)
  table = shape.table

  #assign table style
  tbl =  shape._element.graphic.graphicData.tbl
  style_id = '{C083E6E3-FA7D-4D7B-A595-EF9225AFEA82}'
  tbl[0][-1].text = style_id


  # Set column names
  for col, column_name in enumerate(df.columns):
      cell = table.cell(0, col)
      cell.text = column_name
      cell.text_frame.paragraphs[0].runs[0].font.size = Pt(11)

  # Populate data
  for row in range(rows):
      for col in range(cols):
          cell = table.cell(row + 1, col)
          cell.text = str(df.iloc[row, col])

  # Change font size
  for row in table.rows:
      for cell in row.cells:
          for paragraph in cell.text_frame.paragraphs:
              for run in paragraph.runs:
                  run.font.size = Pt(11)

  prs.save(pptpath) #this overwrites
  print('Presentation saved')

def ql_3band_batch(folder,outshp,outfolder):
  imglist = getfiles(folder,'.tif')
  for imgpath in imglist:
      name = Path(imgpath).stem + '.png'
      outpath = os.path.join(outfolder,name)
      ql_3band(outshp,imgpath,outpath)

def add_slides_batch(img_folder,pptpath):
  imglist = getfiles(img_folder,'.png')

  prs = Presentation(pptpath)
  title_only_slide_layout = prs.slide_layouts[5]

  for i in imglist:
      print(i)
      slide = prs.slides.add_slide(title_only_slide_layout)
      title = slide.shapes.title
      title.width = Cm(35)
      title.height = Cm(3.3)
      title.top = Cm(0)
      title.left = Cm(0)

      #Add title
      name = Path(i).stem
      title.text = name

      slide.shapes.add_picture(
          i, left=Cm(12.78), top=Cm(3.7), width=Cm(12.24), height=None
      )
  prs.save(pptpath)

def inset_map(bc_path,fire_perim,outpath):
  # Define the bounding box for British Columbia (lonmin, lonmax, latmin, latmax)
  bbox = [-139, -114.75, 47.5, 60]


  # Coordinates of cities/towns (latitude, longitude)
  cities = {
      'Vancouver': (49.2827, -123.1207),
      'Kamloops': (50.6761, -120.3408),
      'Prince George':(53.9170,-122.7494),
      'Fort St John': (56.2464,-120.8476),
      'Prince Rupert': (54.3125,-130.3054),
      'Williams Lake': (52.1284,-122.1302)

  }

  # Create a map
  plt.figure(figsize=(4,4))
  ax = plt.axes(projection=ccrs.AlbersEqualArea(central_longitude=-126, central_latitude=54))
  ax.set_extent(bbox, crs=ccrs.PlateCarree())

  # Add land and coastline features
  ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='lightgrey',linewidth=0.1)
  ax.add_feature(cfeature.COASTLINE, linewidth=0.1)

  # Load the British Columbia boundary data using GeoPandas
  ## Thank you GeoBC! https://catalogue.data.gov.bc.ca/dataset/province-of-british-columbia-boundary-terrestrial
  bc_boundary = geopandas.read_file(bc_path)

  # Plot the British Columbia boundary using Cartopy's geopandas tools
  ax.add_geometries(bc_boundary['geometry'], crs=ccrs.PlateCarree(), edgecolor='black', facecolor='green',linewidth=0.1)

  # Add gridlines
  ax.gridlines(draw_labels=True, linestyle='--', color='grey')

  # Add fire boundary
  fire_boundary = geopandas.read_file(fire_perim)
  fire_boundary["centroid"] = fire_boundary["geometry"].centroid
  lat = fire_boundary["centroid"].y
  lon = fire_boundary["centroid"].x
  ax.plot(lon,lat, marker='*', color='purple', markersize=5, transform=ccrs.PlateCarree())
  ax.text(lon + 0.5, lat, 'Fire Location', color='purple', fontsize=8, transform=ccrs.PlateCarree())

  # Add cities/towns
  for city, (lat, lon) in cities.items():
      ax.plot(lon, lat, marker='o', color='black', markersize=3, transform=ccrs.PlateCarree())
      ax.text(lon + 0.5, lat, city, color='black', fontsize=7, transform=ccrs.PlateCarree())

  plt.savefig(outpath, bbox_inches='tight', dpi=300)
  plt.close()

# Calculates burn area by severity class
def zonal_barc(barcpath,firepath,outpath):
  def burnsev_name(x):
      if x == 0:
          return('Unknown')
      elif x == 1:
          return('Unburned')
      elif x == 2:
          return('Low')
      elif x == 3:
          return('Medium')
      elif x == 4:
          return('High')
      else:
          print("Wrong burn severity value!")


  #need to clip to fire perimeter a little tighter than what gee outputs
  basename = barcpath[:-4]
  barcpath_clip = basename + '_clip.tif'

  gdal.Warp(barcpath_clip,barcpath,cutlineDSName=firepath,cropToCutline=True)

  dat = gdal.Open(barcpath_clip)
  band = dat.GetRasterBand(1).ReadAsArray()

  x1,px,x2,x3,x4,x5 = dat.GetGeoTransform()

  nodatavalue = 9
  vals, counts = np.unique(band[band != nodatavalue], return_counts=True)

  df = pd.DataFrame(
      {'class':vals,
        'px_count':counts})


  a = df['px_count'].sum()
  df['perc' ] = (df['px_count'] / a)*100
  df['area_m2'] = df['px_count']*(px*px)
  df['area_ha'] = df['area_m2']*0.0001

  #round to 1 decimal place
  df = df.round(1)
  df['burn_sev'] = df['class'].apply(burnsev_name)

  #reorder columns
  df = df[['class','burn_sev','px_count','area_m2','area_ha','perc']]

  #print(df) #debug
  df.to_csv(outpath)
  return(df)



---



### **Part 2: Google Earth Engine authentication and initialization**

Authenticate and intialize GEE. After running the cell below, a sign-in window will pop-up. Please follow prompts to authenticate (sign-in, continue and continue).

In [6]:
#Authenticate gee
ee.Authenticate()

 Please note, the **Project ID** may be something other than the project name (ex. burn-severity-2024) and may contain additional numbers (ex. burn-severity-2024-456181). Make sure to copy the actual **Project ID** and enter it in the cell below.

In [7]:
# Initialize with a google cloud project
project = 'burn-severity-2024'
ee.Initialize(project=project)



---



### **Part 3: Fire perimeter import and visualization**

The code below will load in a polygon shapefile and display the attribute table. The fire_shp variable is the path to your dataset.

In [8]:
# Open fires shapefile
fires_shp = 'burnSeverity/test/vectors/K70910.shp'
fires = geemap.shp_to_ee(fires_shp)

# Visualize in table format
fires_df = geopandas.read_file(fires_shp)
fires_df

/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Field TRACK_DATE create as date field, though DateTime requested.
  ogr_write(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Field LOAD_DATE create as date field, though DateTime requested.
  ogr_write(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Value 279718852.108900011 of field FEATURE_AR of feature 0 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(


,FIRE_NUMBE,VERSION_NU,FIRE_YEAR,FIRE_SIZE_,SOURCE,TRACK_DATE,LOAD_DATE,FIRE_STATU,FIRE_URL,FEATURE_CO,FEATURE_AR,FEATURE_LE,OBJECTID,SHAPE_AREA,SHAPE_LEN,geometry
0,K70910,2024082001,2024,27971.9,Processed IR image,2024-08-20,2024-08-20,Being Held,https://wildfiresituation.nrs.gov.bc.ca/incide...,JA70003000,2.797189e+08,137780.3225,9165357,0.0,0.0,"POLYGON ((1320010.372 620243.186, 1320041.101 ..."


Now visualize spatially.

In [9]:
Map = geemap.Map()
Map.addLayer(fires,{},'Fire Polys')
Map.centerObject(fires,zoom=8)
Map

Map(center=[50.5247667490912, -121.40723329369156], controls=(WidgetControl(options=['position', 'transparent_…



---



### **Part 4: Individual fire perimeter selection (1 fire perimeter)**

Select one fire perimeter by fire number. Please make sure that the **fieldname** variable matches your dataset.

In [10]:
# Now select one fire (in the test data, there's only one fire perimeter)
firenumber = 'K70910' #change fire name
fieldname = 'FIRE_NUMBE' #unique firenumber field, change if needed

# First check if the firenumber exists in the shapefile provided
firelist = fires_df[fieldname].tolist()

if firenumber not in firelist:
  print('Selected fire number:',firenumber)
  print('Available fire numbers: ',firelist)
  raise ValueError('Fire number not in fire list. Typo?')

# Create output folder
if not os.path.exists(firenumber):
  os.mkdir(firenumber)

# Save a copy of the fire perimeter
vector_folder = os.path.join(firenumber,'vectors')
if not os.path.exists(vector_folder):
  os.mkdir(vector_folder)

outshp = os.path.join(vector_folder,firenumber+'.shp')
fires_df_sub = fires_df[fires_df[fieldname]==firenumber]
fires_df_sub.to_file(outshp,driver='ESRI Shapefile')

# Load in the single perimeter
poly = geemap.shp_to_ee(outshp)




---



### **Part 5: Sensor selection**

 The sensor options are as follows: Sentinel-2a/b MSI (S2), Landsat-8 OLI (L8) or Landsat-9 OLI-2 (L9). The data from the above sensors are the surface reflectance / bottom of atmosphere processing level. Pre- and post-fire imagery wil be selected from the same sensor.

In [11]:
dattype = 'S2' #change sensor here

if dattype == 'S2':
    col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    cld_field =  'CLOUDY_PIXEL_PERCENTAGE'
    print('Selected S2 SR')
elif dattype == 'L9':
    col = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
    cld_field = 'CLOUD_COVER'
    print('Selected L9 SR')
elif dattype == 'L8':
    col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    cld_field = 'CLOUD_COVER'
    print('Selected L8 SR')
else:
    raise ValueError('Wrong data type selected. Choose S2, L9 or L8.')


Selected S2 SR


**Visualization parameters**: choose the type of visualization you would like to use. Default is false-colour infrared ('nir') but true-color ('tc') and shortwave-infrared ('swir') are also available.

In [12]:
# Define visualization parameters
vis_type = 'nir'

# Dictionary with band combinations
vis_dict = {'L8':{'nir':['SR_B5', 'SR_B4', 'SR_B3'],
                  'tc':['SR_B4','SR_B3','SR_B2'],
                  'swir':['SR_B7','SR_B6','SR_B4']},
            'L9':{'nir':['SR_B5', 'SR_B4', 'SR_B3'],
                  'tc':['SR_B4','SR_B3','SR_B2'],
                  'swir':['SR_B7','SR_B6','SR_B4']},
            'S2':{'nir':['B8', 'B4', 'B3'],
                  'tc':['B4','B3','B2'],
                  'swir':['B12','B8','B4']}}

bands = vis_dict[dattype][vis_type] #or provide a list of 3 bands

vis = {
  'min': 0,
  'max': 0.3,
  'gamma':1.5,
  'bands': bands,
};



---



### **Part 6: Pre-fire image search, visualization and selection**

Select the time-interval for the pre-fire image search (startdate_pre and enddate_pre). The imagery should be selected either from year of the fire or the year before. The optimal window for image selection is July 1st to September 30. If good quality imagery isn't available, then search previous year's imagery. To avoid phenology differences, it is best to select pre- and post-fire imagery that is as seasonally consistent as possible. For example both and pre- and post-fire imagery acquired in early August.

The script below will search the archive for images acquired during the defined time interval (not inclusive of the enddate), overlapping with the fire perimeter, and below the defined cloud cover threshold. The resulting images are then mosaicked by image acquisition day (along track). If you are searching a wide range time range, it is best to reduce the cloud threshold (cld_thr_pre) to 40 or less.


In [15]:
# Find pre-fire imagery
startdate_pre = '2024-07-05'
enddate_pre = '2024-07-15'
cld_thr_pre = 100 #this is the cloud cover threshold from the scene metadata

before = col.filterDate(startdate_pre,enddate_pre).filterBounds(poly).filter(ee.Filter.lt(cld_field,cld_thr_pre))

before_list = before.toList(before.size().getInfo())
pre_mosaic_col = runDateMosaic(before_list)

if dattype.startswith('L'):
  pre_mosaic_col = pre_mosaic_col.map(apply_scale_factors_ls)
else:
  pre_mosaic_col = pre_mosaic_col.map(apply_scale_factors_s2)

print('Found',pre_mosaic_col.size().getInfo(),'dates')

Found 4 dates


**Pre-fire image map display:** display the imagery found.

In [16]:
# Add pre-fire images to pre-fire map
before_map = geemap.Map()

before_size = pre_mosaic_col.size().getInfo()
before_mosaic_list = pre_mosaic_col.toList(before_size)
before_size

for i in range(0,before_size):
  b = before_mosaic_list.get(i)
  date = b.getInfo()['properties']['system:index']
  before_map.addLayer(ee.Image(b), vis, date)
  print(date)

# Add fire polygon
style = {'color': 'white', 'width': 2, 'lineType': 'solid', 'fillColor': '00000000'}
before_map.addLayer(poly.style(**style),{},'poly')
before_map.centerObject(poly,zoom=8)
before_map

2024-07-05
2024-07-08
2024-07-10
2024-07-13


Map(center=[50.5247667490912, -121.40723329369156], controls=(WidgetControl(options=['position', 'transparent_…

In the top right corner of the map, select the option to list all the layers and toggle on and off the dates to inspect the imagery. Find a high quality image, free of cloud, smoke, as well as snow and take note of the date(yyyy-mm-dd). You can copy and paste the date from the output of the cell above. If there's no clear imagery available for the date you range you defined you can try a different date range or go back and select a different sensor (dattype).



---



### **Part 7: Post-fire image search, visualization and selection**

Now select the time-interval for the post-fire image search (startdate_post and enddate_post). As with the pre-fire imagery, the optimal window for image selection is July 1st to September 30. However, that will not be possible, especially for the fires later in the season. If possible, try to select imagery from before October 15th.

In [17]:
# Find post-fire images
startdate_post = '2024-09-05'
enddate_post = '2024-09-10' #not inclusive of the end date
cld_thr_post = 100

after = col.filterDate(startdate_post,enddate_post).filterBounds(poly).filter(ee.Filter.lt(cld_field,cld_thr_post))

after_list = after.toList(after.size().getInfo())
post_mosaic_col = runDateMosaic(after_list)

if dattype.startswith('L'):
  post_mosaic_col = post_mosaic_col.map(apply_scale_factors_ls)
else:
  post_mosaic_col = post_mosaic_col.map(apply_scale_factors_s2)

print('Found',post_mosaic_col.size().getInfo(),'scenes')

Found 2 scenes


**Post-fire image map display**: display the post-fire imagery in a new map.

In [19]:
# Add post-fire images to post-fire map
after_map = geemap.Map()

post_size = post_mosaic_col.size().getInfo()
post_mosaic_list = post_mosaic_col.toList(post_size)

for i in range(0,post_size):
  b = post_mosaic_list.get(i)
  date = b.getInfo()['properties']['system:index']
  after_map.addLayer(ee.Image(b), vis, date)
  print(date)

# Add fire polygon
style = {'color': 'white', 'width': 2, 'lineType': 'solid', 'fillColor': '00000000'}
after_map.addLayer(poly.style(**style),{},'poly')
after_map.centerObject(poly,zoom=8)
after_map


2024-09-06
2024-09-08


Map(center=[50.5247667490912, -121.40723329369156], controls=(WidgetControl(options=['position', 'transparent_…

Also take note of the post-fire image selected. If good quality post-fire imagery is not available consider changing the date range or the sensor.



---



### **Part 8: BARC mapping**

The BARC burn severity raster raster will be created below. Change the dates for the pre_mosaic_date and post_mosaic_date variables. The BARC raster is generated by: (1) calculating pre- and post-NBR, (2) calculating dNBR by subtracting postNBR from preNBR, (3) Applying a scaling equation to dNBR and (4) Applying thresholds to create a 4-class burn severity raster (1-unburned, 2-low, 3-medium, 4-high).

In [23]:
# Calculate NBR, dNBR and generate BARC map.
pre_mosaic_date = '2024-07-10' #enter pre-fire image date here
post_mosaic_date = '2024-09-06' #enter post_fire image date here

# Select pre-image and post-image
pre_col = pre_mosaic_col.filter(ee.Filter.inList("system:index",ee.List([pre_mosaic_date])))
pre_img = ee.Image(pre_col.toList(1).get(0))

if pre_col.size().getInfo() != 1:
  raise ValueError("Didn't select 1 pre-fire image date. Check pre_mosaic_date!")

post_col = post_mosaic_col.filter(ee.Filter.inList("system:index", ee.List([post_mosaic_date])))
post_img = ee.Image(post_col.toList(1).get(0))

if post_col.size().getInfo() != 1:
  raise ValueError("Didn't select 1 post-fire image date. Check post_mosaic_date!")

# Calculate NBR
if dattype.startswith('S2'):
    pre_nbr = NBR_S2(pre_img)
    post_nbr = NBR_S2(post_img)
else:
    pre_nbr = NBR_Landsat(pre_img,dattype)
    post_nbr = NBR_Landsat(post_img,dattype)

# Calculate dNBR
dNBR = pre_nbr.subtract(post_nbr).rename('dNBR')

#FAIB: scaling for static dNBR thresholding (0.105, 0.275, 0.660)
dNBR_scaled = dNBR.expression('(dNBR * 1000 + 275)/5',{'dNBR': dNBR.select('dNBR')}).rename('dNBR_scaled')

#BC Wildfire: multiplied by 1000 and converted to integer. FAIB thresholds would be 105,275,660
dNBR_continuous = dNBR.multiply(1000).toInt16()

# Classify
classes = dNBR_scaled.expression("(dNBR_scaled >= 187) ? 4 "
                                ": (dNBR_scaled >= 110) ? 3 "
                                ": (dNBR_scaled >= 76) ? 2 "
                                ": 1",{'dNBR_scaled': dNBR_scaled.select('dNBR_scaled')})

#TODO:Mask out water, set to 9, which will become no data
# esa_water = ee.ImageCollection('ESA/WorldCover/v200').filterBounds(poly).first().eq(80);

#TODO: implement cloud masking

# Clip to fire perimeter
classes_clipped = classes.clip(poly)

print("BARC Complete")

# Get scene ids to save
d = ee.Date(pre_mosaic_date)
pre_filt = before.filterDate(d,d.advance(1,'day'))
pre_scenes_ids = pre_filt.aggregate_array('system:index').getInfo()
print(pre_scenes_ids)

d1 = ee.Date(post_mosaic_date)
post_filt = after.filterDate(d1,d1.advance(1,'day'))
post_scenes_ids = post_filt.aggregate_array('system:index').getInfo()
print(post_scenes_ids)

BARC Complete
['20240710T185919_20240710T190216_T10UEA', '20240710T185919_20240710T190216_T10UEB', '20240710T185919_20240710T190216_T10UFA', '20240710T185919_20240710T190216_T10UFB']
['20240906T190921_20240906T191124_T10UEA', '20240906T190921_20240906T191124_T10UEB', '20240906T190921_20240906T191124_T10UFA', '20240906T190921_20240906T191124_T10UFB']




---



### **Part 9: Final visualization for quality control**

This script is the final visualization of pre- and post-fire imagery as well as the BARC map. To assess the quality, toggle the layers and zoom into different regions to make sure that the pre- and post-fire imagery is free of clouds, smoke, haze or active fire.

In [ ]:
#Visualize pre- and post-fire imagery with BARC
barc_map = geemap.Map()
barc_map.addLayer(ee.Image(pre_img), vis, pre_mosaic_date)
barc_map.addLayer(ee.Image(post_img),vis, post_mosaic_date)

palette = ['000000','grey', 'yellow', 'orange','red']
keys = ['Unknown','Unburned','Low','Medium','High']
colors = [(0, 0, 0),(128, 128, 128),(255, 255, 0),(255, 165, 0), (255, 0, 0)]

barc_vis = {'min':0,'max':4,'palette':palette}
barc_map.addLayer(ee.Image(classes_clipped),barc_vis,'barc')
barc_map.add_legend(keys=keys,colors=colors,position='bottomright')

style = {'color': 'white', 'width': 2, 'lineType': 'solid', 'fillColor': '00000000'}
barc_map.addLayer(poly.style(**style),{},'poly')

barc_map.centerObject(poly)
barc_map

### **Part 10: Image download**

This script will download a BARC raster clipped to the extent of the fire perimeter polygon, as well as pre- and post-fire imagery (true color and swir RGB, 8-bit). The search criteria and scene ids will also be exported as json and text files. The root folder will be the fire number.

In [ ]:
# Export
outfolder = firenumber #root folder
pre_date = pre_mosaic_date.replace('-','')
post_date = post_mosaic_date.replace('-','')

# Delete folder if exists and re-create
barc_folder = os.path.join(outfolder,'barc')
if os.path.exists(barc_folder):
  print(barc_folder,'exists. Deleting.')
  shutil.rmtree(barc_folder)
os.makedirs(barc_folder)

pre_tc_8bit = os.path.join(outfolder,'pre_truecolor_8bit')
if os.path.exists(pre_tc_8bit):
  print(pre_tc_8bit,'exists. Deleting.')
  shutil.rmtree(pre_tc_8bit)
os.makedirs(pre_tc_8bit)

post_tc_8bit = os.path.join(outfolder,'post_truecolor_8bit')
if os.path.exists(post_tc_8bit):
  print(post_tc_8bit,'exists. Deleting.')
  shutil.rmtree(post_tc_8bit)
os.makedirs(post_tc_8bit)

pre_sw_8bit = os.path.join(outfolder,'pre_swir_8bit')
if os.path.exists(pre_sw_8bit):
  print(pre_sw_8bit,'exists. Deleting.')
  shutil.rmtree(pre_sw_8bit)
os.makedirs(pre_sw_8bit)

post_sw_8bit = os.path.join(outfolder,'post_swir_8bit')
if os.path.exists(post_sw_8bit):
  print(post_sw_8bit,'exists. Deleting.')
  shutil.rmtree(post_sw_8bit)
os.makedirs(post_sw_8bit)

# Define spatial resolution
if dattype == 'S2':
  px = 20
else:
  px = 30

## Define tiling rules
poly_area = round(poly.geometry().area(1).divide(10000).getInfo(),1)
print(poly_area)
print('Fire Area:',poly_area,'hectares')

if poly_area < 10000:
    n = 2
elif poly_area > 10000 and poly_area < 100000:
    n = 3
elif poly_area > 100000 and poly_area < 400000:
    n = 4
else:
    n = 5

print('Number of tiles: ' + str(n*n))

#export pre and post rgbs, tile to avoid pixel limit issues.
footprint = poly.geometry().bounds().getInfo()
grids = grid_footprint(footprint,n,n)

for i in range(0,len(grids)):
  roi = grids[i]
  ## Export BARC
  barc_folder = os.path.join(outfolder,'barc')
  if not os.path.exists(barc_folder):
          os.makedirs(barc_folder)

  name = 'BARC_' + firenumber + '_' + pre_date + '_' + post_date + '_' + dattype +'_' + str(i) + '_.tif'
  barc_filename = os.path.join(barc_folder,name)
  geemap.ee_export_image(classes_clipped.unmask(9).clip(roi), filename=barc_filename, scale=px, file_per_band=False,crs='EPSG:3005')
  ras = gdal.Open(barc_filename,gdal.GA_Update)
  dat = ras.GetRasterBand(1)
  dat.SetNoDataValue(9)
  ras = None
  dat = None

  ## Export continuous dNBR (dNBR_continuous)
  dNBR_folder = os.path.join(outfolder,'dNBR')
  if not os.path.exists(dNBR_folder):
          os.makedirs(dNBR_folder)

  name = 'dNBR_' + firenumber + '_' + pre_date + '_' + post_date + '_' + dattype +'_' + str(i) + '_.tif'
  dNBR_filename = os.path.join(dNBR_folder,name)
  if dattype.startswith('S2'):
    geemap.ee_export_image(dNBR_continuous.clip(roi), filename=dNBR_filename, scale=20, file_per_band=False,crs='EPSG:3005')
  elif (dattype == 'L8') | (dattype == 'L9'):
    geemap.ee_export_image(dNBR_continuous.clip(roi), filename=dNBR_filename, scale=30, file_per_band=False,crs='EPSG:3005')
  else:
    pass

  ## Export 8-bit truecolor images
  #pre, truecolor
  pre_tc_8bit = os.path.join(outfolder,'pre_truecolor_8bit')
  if not os.path.exists(pre_tc_8bit):
          os.makedirs(pre_tc_8bit)
  filename = os.path.join(pre_tc_8bit, dattype + '_' + pre_date + '_truecolor_pre_8bit_' + str(i) + '.tif')
  pre_tc_8bit_path = filename
  if dattype.startswith('S2'):
      viz = {'bands': ['B4', 'B3', 'B2'], 'min': -0.01, 'max':0.3,'gamma':1.5}
      geemap.ee_export_image(pre_img.clip(roi).visualize(**viz), filename=filename, scale=10, file_per_band=False,crs='EPSG:3005')
  elif (dattype == 'L8') | (dattype == 'L9'):
      viz = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': -0.01, 'max':0.3,'gamma':1.5}
      geemap.ee_export_image(pre_img.clip(roi).visualize(**viz), filename=filename, scale=30, file_per_band=False,crs='EPSG:3005')
  else:
      pass

  post_tc_8bit = os.path.join(outfolder,'post_truecolor_8bit')
  if not os.path.exists(post_tc_8bit):
          os.makedirs(post_tc_8bit)
  filename = os.path.join(post_tc_8bit, dattype + '_' + post_date + '_truecolor_post_8bit_' + str(i) + '.tif')
  post_tc_8bit_path = filename
  if dattype.startswith('S2'):
      viz = {'bands': ['B4', 'B3', 'B2'], 'min': -0.01, 'max':0.3,'gamma':1.5}
      geemap.ee_export_image(post_img.clip(roi).visualize(**viz), filename=filename, scale=10, file_per_band=False,crs='EPSG:3005')
  elif (dattype == 'L8') | (dattype == 'L9'):
      viz = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': -0.01, 'max':0.3,'gamma':1.5}
      geemap.ee_export_image(post_img.clip(roi).visualize(**viz), filename=filename, scale=30, file_per_band=False,crs='EPSG:3005')
  else:
      pass

  ## Export swir too
  pre_sw_8bit = os.path.join(outfolder,'pre_swir_8bit')
  if not os.path.exists(pre_sw_8bit):
          os.makedirs(pre_sw_8bit)
  filename = os.path.join(pre_sw_8bit, dattype + '_' + pre_date + '_swir_pre_8bit_' + str(i) + '.tif')
  pre_sw_8bit_path = filename
  if dattype.startswith('S2'):
      viz = {'bands': ['B12', 'B8', 'B4'], 'min': -0.01, 'max':0.3,'gamma':1.5}
      geemap.ee_export_image(pre_img.clip(roi).visualize(**viz), filename=filename, scale=10, file_per_band=False,crs='EPSG:3005')
  elif (dattype == 'L8') | (dattype == 'L9'):
      viz = {'bands': ['SR_B6', 'SR_B5', 'SR_B4'], 'min': -0.01, 'max':0.3,'gamma':1.5}
      geemap.ee_export_image(pre_img.clip(roi).visualize(**viz), filename=filename, scale=30, file_per_band=False,crs='EPSG:3005')
  else:
      pass

  post_sw_8bit = os.path.join(outfolder,'post_swir_8bit')
  if not os.path.exists(post_sw_8bit):
          os.makedirs(post_sw_8bit)
  filename = os.path.join(post_sw_8bit, dattype + '_' + post_date + '_swir_post_8bit_' + str(i) + '.tif')
  post_sw_8bit_path = filename
  if dattype.startswith('S2'):
      viz = {'bands': ['B12', 'B8', 'B4'], 'min': -0.01, 'max':0.3,'gamma':1.5}
      geemap.ee_export_image(post_img.clip(roi).visualize(**viz), filename=filename, scale=10, file_per_band=False,crs='EPSG:3005')
  elif (dattype == 'L8') | (dattype == 'L9'):
      viz = {'bands': ['SR_B6', 'SR_B5', 'SR_B4'], 'min': -0.01, 'max':0.3,'gamma':1.5}
      geemap.ee_export_image(post_img.clip(roi).visualize(**viz), filename=filename, scale=30, file_per_band=False,crs='EPSG:3005')
  else:
      pass

  print(barc_folder)

#mosaic all
#BARC
barc_list = getfiles(barc_folder,'.tif')
outfilename = 'BARC_' + firenumber + '_' + pre_date + '_' + post_date + '_' + dattype + '.tif'
out = os.path.join(barc_folder,outfilename)
gdal.Warp(out,barc_list)
for file in barc_list: os.remove(file) #delete tiles
print('Barc mosaic complete')

#dNBR
dNBR_list = getfiles(dNBR_folder,'.tif')
outfilename = 'dNBR_' + firenumber + '_' + pre_date + '_' + post_date + '_' + dattype + '.tif'
out = os.path.join(dNBR_folder,outfilename)
gdal.Warp(out,dNBR_list)
for file in dNBR_list: os.remove(file) #delete tiles

#pre truecolour
pre_tc_list = getfiles(pre_tc_8bit,'.tif')
outfilename = dattype + '_' + pre_date + '_truecolor_pre_8bit' + '.tif'
out = os.path.join(pre_tc_8bit,outfilename)
gdal.Warp(out,pre_tc_list)
for file in pre_tc_list: os.remove(file) #delete tiles
pre_tc_8bit_path = out #to return from the function
print('Pre truecolor mosaic complete')

#post truecolor
post_tc_list = getfiles(post_tc_8bit,'.tif')
outfilename = dattype + '_' + post_date + '_truecolor_post_8bit' + '.tif'
out = os.path.join(post_tc_8bit,outfilename)
gdal.Warp(out,post_tc_list)
for file in post_tc_list: os.remove(file)
post_tc_8bit_path = out #to return from the function
print('Post truecolor mosaic complete')

#pre swir
pre_sw_list = getfiles(pre_sw_8bit,'.tif')
outfilename = dattype + '_' + pre_date + '_swir_pre_8bit' + '.tif'
out = os.path.join(pre_sw_8bit,outfilename)
gdal.Warp(out,pre_sw_list)
for file in pre_sw_list: os.remove(file) #delete tiles
pre_sw_8bit_path = out #to return from the function
print('Pre swir mosaic complete')

#post swir
post_sw_list = getfiles(post_sw_8bit,'.tif')
outfilename = dattype + '_' + post_date + '_swir_post_8bit' + '.tif'
out = os.path.join(post_sw_8bit,outfilename)
gdal.Warp(out,post_sw_list)
for file in post_sw_list: os.remove(file)
post_sw_8bit_path = out #to return from the function
print('Post swir mosaic complete')

#Export a dictionary with metadata info
searchd = {'Id':firenumber,'sensor':dattype,
               'cld_pre':cld_thr_pre,'pre_T1':startdate_pre,'pre_T2':enddate_pre,
               'cld_post':cld_thr_post,'post_T1':startdate_post,'post_T2':startdate_post,
               'pre_mosaic_date':pre_mosaic_date,'pre_scenes':pre_scenes_ids,
               'post_mosaic_date':post_mosaic_date,'post_scenes':post_scenes_ids}

params = os.path.join(outfolder,'search_params.txt')
with open(params, 'w') as f:
    for key, value in searchd.items():
        f.write('%s:%s\n' % (key, value))

#write search parameters to the folder as json for use later
output_json = os.path.join(outfolder,'search_params.json')
with open(output_json, 'w') as json_file:
    json.dump(searchd, json_file, indent=4)


27873.8
Fire Area: 27873.8 hectares
Number of tiles: 9
Generating URL ...
Please wait ...
Data downloaded to /content/K70910/barc/BARC_K70910_20240710_20240906_S2_0_.tif
Generating URL ...
Please wait ...
Data downloaded to /content/K70910/dNBR/dNBR_K70910_20240710_20240906_S2_0_.tif
Generating URL ...
Please wait ...
Data downloaded to /content/K70910/pre_truecolor_8bit/S2_20240710_truecolor_pre_8bit_0.tif
Generating URL ...
Please wait ...
Data downloaded to /content/K70910/post_truecolor_8bit/S2_20240906_truecolor_post_8bit_0.tif
Generating URL ...
Please wait ...
Data downloaded to /content/K70910/pre_swir_8bit/S2_20240710_swir_pre_8bit_0.tif
Generating URL ...
Please wait ...
Data downloaded to /content/K70910/post_swir_8bit/S2_20240906_swir_post_8bit_0.tif
K70910/barc
Generating URL ...
Please wait ...
Data downloaded to /content/K70910/barc/BARC_K70910_20240710_20240906_S2_1_.tif
Generating URL ...
Please wait ...
Data downloaded to /content/K70910/dNBR/dNBR_K70910_20240710_2024

### **Part 11: Quicklooks and area burned by severity class**

The script below creates and exports quicklook images, burn severity classes summaries and maps. You should be able to run as is.

In [ ]:
# Summary Section
qcdir = os.path.join(firenumber,'QC')
if os.path.exists(qcdir):
  print(qcdir,'exists. Deleting.')
  shutil.rmtree(qcdir)
os.mkdir(qcdir)

# Create powerpoint presentation
pptpath = os.path.join(qcdir,firenumber+'.pptx')
create_ppt(pptpath)

bc_boundary ='burnSeverity/bc/BC_Boundary_Terrestrial_gcs_simplify.shp'

# Pre- and post-fire quicklooks (truecolor)
pre_tc_ql = generate_ql(outshp,pre_tc_8bit_path,qcdir)
post_tc_ql = generate_ql(outshp,post_tc_8bit_path,qcdir)

# Pre- and post-fire quicklooks (swir)
pre_sw_ql = generate_ql(outshp,pre_sw_8bit_path,qcdir)
post_sw_ql = generate_ql(outshp,post_sw_8bit_path,qcdir)

# BARC
name = Path(barc_filename).stem + '.png'
barc_ql = os.path.join(qcdir,name)
ql_barc(outshp,barc_filename,post_tc_8bit_path,barc_ql)

# Location map
name = firenumber + '_locmap.png'
map_ql = os.path.join(qcdir,name)

fire_perim = os.path.join(firenumber,'vectors',firenumber+'_gcs.shp')
if os.path.isfile(fire_perim):
    pass
else:
    fire_perim = os.path.join(firenumber,'vectors',firenumber+'.shp')

inset_map(bc_boundary,fire_perim,map_ql)

#add stats table
outpath = os.path.join(firenumber,'barc_stats.csv')
df = zonal_barc(barc_filename,outshp,outpath)

#Add slide to powerpoint
add_slide(pptpath,pre_tc_ql,post_tc_ql,barc_ql,map_ql,df)
add_slide(pptpath,pre_sw_ql,post_sw_ql,barc_ql,map_ql,df)





---



### **Part 12: Final export**

The folder will be zipped and saved under the Files tab. Right click and download the folder to your machine.

In [ ]:
#Zip to download
zip = firenumber + '.zip'
indata = firenumber

!zip -r {zip} {indata}
files.download(zip)